In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import scipy.signal as scsig

In [ ]:
import data_processing as processor
from data_handling import LaserDataInteractorESRF

In [ ]:
processNo = 18
chNo = 1

path_rel = '/Volumes/Sandisk_SD/Work/IZFP/Laser/ESRF/Code/measurements/ESRF_TU_Ilmenau_IZFP/Messdaten_QASS'
#path_rel = '/Volumes/Samsung_SD/temp/ESRF_TU_Ilmenau_IZFP/Messdaten_QASS'

In [ ]:
interactor = LaserDataInteractorESRF()
interactor.path_rel = path_rel
interactor.fname = (chNo, processNo)

data, dt = interactor.load_data(ret_dt=True)

In [ ]:
#%matplotlib widget
%matplotlib inline

#plt.rcParams['figure.dpi'] = 150

# Spectrogram
fig, (ax1, ax3, ax4) = plt.subplots(3, 1)
# Figure size
fig.set_figwidth(10)
fig.set_figheight(8)

# param
nfft = 512

ax1.specgram(data, NFFT=nfft, Fs=1/dt, noverlap=int(nfft/2), cmap='rainbow')
# xaxis
ax1.set_xticklabels([])
#yaxis
yticks = np.arange(0, 0.5/dt, 0.5*10**6)
ax1.set_yticks(yticks)
ax1.set_yticklabels(np.around(10**-3* yticks).astype(int))
ax1.set_ylabel('f [kHz]')
ax1.set_title('Whole frequency range')
ax1.grid(True)

# # Limiting the frequency range: middle
# ax2.specgram(data, NFFT=nfft, Fs=1/dt, noverlap=int(nfft/2), cmap='rainbow')
# # xaxis
# ax2.set_xticklabels([])
# # yaxis
# ymin = 0*10**6#None
# ymax = 0.51*10**6
# ax2.set_ylim(bottom =ymin, top=ymax)
# yticks = np.arange(ymin, ymax, 10**5)
# ax2.set_yticks(yticks)
# ax2.set_yticklabels(np.around(10**-3* yticks).astype(int))
# ax2.set_ylabel('f [kHz]')
# ax2.set_title('Low frequency range')
# ax2.grid(True)


# Limiting the frequency range: middle
ax3.specgram(data, NFFT=nfft, Fs=1/dt, noverlap=int(nfft/2), cmap='rainbow')
# xaxis
ax3.set_xticklabels([])
# yaxis
ymin = 0.25*10**6#None
ymax = 0.45*10**6
ax3.set_ylim(bottom =ymin, top=ymax)
yticks = np.arange(ymin, ymax, 0.5*10**5)
ax3.set_yticks(yticks)
ax3.set_yticklabels(np.around(10**-3* yticks).astype(int))
ax3.set_ylabel('f [kHz]')
ax3.set_title('Middle frequency range')
ax3.grid(True)


# Limiting the frequency range: middle
ax4.specgram(data, NFFT=nfft, Fs=1/dt, noverlap=int(nfft/2), cmap='rainbow')
# yaxis
ymin = 15*10**3#[Hz]
ymax = 200*10**3#[Hz]
ax4.set_ylim(bottom =ymin, top=ymax)
yticks = np.arange(ymin, ymax, 0.5*10**5)
ax4.set_yticks(yticks)
ax4.set_yticklabels(np.around(10**-3* yticks).astype(int))
ax4.set_ylabel('f [kHz]')
ax4.set_title('Low frequency range')
ax4.grid(True)

# xaxis
ax4.set_xlabel('t [s]')

plt.suptitle(
    f'Data No.901 (= QASS No.{processNo}), spectrgoram'
)
plt.show()

## Processing

In [ ]:
# processing
smoothing = 0.05*10**-3 #[s], window duration for smoothing
fmin = 20*10**3 # [Hz]
fmax = 200*10**3 # [Hz]

data_proc = processor.process_laser_data(
    sig=data, 
    f_range=[fmin, fmax], #[Hz] 
    dt=dt, 
    w_duration=smoothing
)


## Synchronization
What we know so far:
* We know the HD recording time for the followings
    * oben = d1 = 0.604mm -> t1
    * mitte = d2 = 1.604mm -> t2
    * unten = d3 = 2.604mm -> t3
* We measured the penetration depth of each hole
    * penetration depth = d_pen = d4 is known with some deviation (vary with each hole)
    * drilling duration T_drill = d4 / v_drill
    * the finishing point of the drilling, t4, should be calcluated 
        * t4 = t1 + (d4 - d1)/v_drill
        * t4 = t2 + (d4 - d2)/v_drill
        * t4 = t3 + (d4 - d3)/v_drill
* In theory, the starting point of the drilling, t0, should be able to calcluated as 
    * t0 = t4 - T_drill

In [ ]:
# HD video info
# 3 drilling time points: d1 = 0.604mm depth, d2 = 1.604mm depth, d3 = 2.604mm depth
d1 = 0.604 #[mm]
d2 = 1.604 #[mm]
d3 = 2.604 #[mm]
# Load the HD frame info -> convert to second
fs_HD = 40*10**3 # [Hz]
t1 = np.loadtxt(f'{path_rel}/901_pulse/frame_oben.txt')/fs_HD #[s], shape = 37
t2 = np.loadtxt(f'{path_rel}/901_pulse/frame_mitte.txt')/fs_HD #[s], shape = 37
t3 = np.loadtxt(f'{path_rel}/901_pulse/frame_unten.txt')/fs_HD #[s], shape = 37

In [ ]:
# Drilling penetration depth and duration
def load_and_convert(path):
    """
    In the Exceltable, comma is used for decimal point, which cannot be read with np.load
    -> we need to 
        (1) load the data as strings
        (2) replace the comma with the periodo
        (3) store as a float number in an array
    """

    with open(path) as f:
        contents = f.readlines()

    arr =np.zeros(len(contents)-1)
    for idx, line in enumerate(contents[1:]):
        s = line.strip('\n')
        s = s.strip().replace(',', '.')
        arr[idx] = s
    return arr

d4 = load_and_convert(f'{path_rel}/901_pulse/drill_depth.txt') #[mm], shape = 37
T_drill = load_and_convert(f'{path_rel}/901_pulse/drill_duration.txt')*10**-3 #[s], shape = 37  
v_drill = load_and_convert(f'{path_rel}/901_pulse/drill_speed.txt') #[mm/s], shape = 37  

In [ ]:
# Starting point of the drill -> for synchronization?
# Computing t4 in 3 different ways
t4_all = np.zeros((len(d4), 3))
t4_all[:, 0] = t1 + (d4 - d1)/v_drill # using t1
t4_all[:, 1] = t2 + (d4 - d2)/v_drill # using t2
t4_all[:, 2] = t3 + (d4 - d3)/v_drill # using t3
# Averaging
t4 = np.mean(t4_all, axis=1)
# Compute t0
t0 = t4 - T_drill

In [ ]:
%matplotlib inline
plt.plot((t1-t0)*10**3)
plt.title('t1 - t0 for all holes')
plt.ylabel('$\Delta_{t}$ [ms]')
plt.show()

In [ ]:
# There may be delay b/w the systems -> synchronization is required?
buffer = round(-80.7*10**-3, 5) #[s]

# Plot

In [ ]:
# Plot each segment
# Trim the signal for display
t_offset = 17.0*10**-3
T_pulse = 11*10**-3 # [s], interval between two laser pulses
pulses = np.array([20, 21]) # idx, which pulse we want to see
start = round(t_offset + pulses[0]* T_pulse, 5) #[s]
end = round(t_offset + pulses[1]* T_pulse, 5) #[s]
idx_start, idx_end = (int(start/dt), int(end/dt))
# Pulse handling
# Select the range
idx_pulse = np.argwhere(np.logical_and((t0+buffer > start), (t0+buffer < end))).flatten()
print(idx_pulse)

vmin, vmax = [data_proc[idx_start:idx_end].min()-5, data_proc[idx_start:idx_end].max()+5]


#%matplotlib widget
%matplotlib inline

fig = plt.figure()
# Figure size
fig.set_figwidth(10)
fig.set_figheight(6)


#plt.plot(dt* np.arange(idx_start, idx_end), data[idx_start:idx_end], label = 'raw')
plt.plot(dt* np.arange(idx_start, idx_end), data_proc[idx_start:idx_end], label = 'processed')
# Add the pulse time points
plt.vlines(x=t0[idx_pulse[0]:idx_pulse[-1]+1]+buffer, 
           ymin=vmin, ymax=vmax,
           label='t0',
           colors='C1', alpha=0.7)
plt.vlines(x=t1[idx_pulse[0]:idx_pulse[-1]+1]+buffer, 
           ymin=vmin, ymax=vmax,
           label='t1',
           colors='red', linestyles='dotted')
plt.vlines(x=t2[idx_pulse[0]:idx_pulse[-1]+1]+buffer, 
           ymin=vmin, ymax=vmax,
           label='t2',
           colors='red', linestyles='dashdot')
plt.vlines(x=t3[idx_pulse[0]:idx_pulse[-1]+1]+buffer, 
           ymin=vmin, ymax=vmax,
           label='t3',
           colors='red', linestyles='dashed')
plt.vlines(x=t4[idx_pulse[0]:idx_pulse[-1]+1]+buffer, 
           ymin=vmin, ymax=vmax,
           label='t4',
           colors='green')
# plt.vlines(x=t_p3[idx_pulse[0]:idx_pulse[-1]+1]+t_pdelta[idx_pulse[0]:idx_pulse[-1]+1]+buffer, 
#            ymin=vmin, ymax=vmax,
#            colors='C1', alpha=0.7)


plt.title(
    f'Ch.{chNo}, pocessed data vs pulse position (added buffer = {round(buffer*10**3, 2)}ms) \n' +
    f't = {round(start*10**3, 1)}ms ... {round(end*10**3, 1)}ms, pulses = {pulses[0]+1}...{pulses[1]} \n' +
    f'f = {round(fmin*10**-3)}...{round(fmax*10**-3)}kHz, smoothing window = {smoothing*10**3}ms'
)
plt.xlabel('t [s]')
plt.ylabel('amplitude')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
def pulse_array(vec, N, n_rows=37, offset=1.0*10**-3):
    # Base of the array formatting of the raw data
    arr = np.zeros((n_rows, N))

    for idx, t in enumerate(t0):
        # For trimming and aligning the signal segment 
        middle = t + buffer
        start = middle - offset #[s]
        end = start + round(dt* N, 4) # [s]
        idx_start = int(start/dt)
        idx_end = idx_start + N

        arr[idx, :] = vec[np.newaxis, idx_start:idx_end]
    
    return arr

In [ ]:
N = int(5*10**-3/dt)
offset = 1.0*10**-3 #[s]
arr = np.zeros((37, N))

for idx, t in enumerate(t0):
    # For trimming and aligning the signal segment 
    middle = t + buffer
    start = middle - offset #[s]
    end = start + round(dt* N, 4) # [s]
    idx_start = int(start/dt)
    idx_end = idx_start + N
    
    arr[idx, :] = data_proc[np.newaxis, idx_start:idx_end]
    
    # plt.plot(dt* np.arange(0, (idx_end - idx_start)), data_proc[idx_start:idx_end])
    # # Add the pulse time points
    # # for t
    # plt.vlines(x = middle - start, 
    #            ymin=vmin, ymax=vmax,
    #            colors='green', alpha=0.7)
    # 
    # plt.vlines(x = middle - start + t_pdelta[idx], 
    #            ymin=vmin, ymax=vmax,
    #            colors='red', linestyles='dashed')
    # plt.vlines(x = middle - start + 2* t_pdelta[idx], 
    #            ymin=vmin, ymax=vmax,
    #            colors='green', alpha=0.7)


    
#%matplotlib widget
%matplotlib inline

# Setting
fig, ax = plt.subplots(1, 1)
# Figure size
fig.set_figwidth(8)
fig.set_figheight(10)

im = ax.imshow(arr, aspect=N/37, cmap='rainbow')
plt.colorbar(im)

# Add the t0 line
p_t0 = int(offset/dt) # pulse position 
ax.vlines(x = p_t0, 
          ymin=-0.5, ymax=36.5,
          colors='red', label ='5.0')
# Add the T_drill line
p_drill = p_t0 + (T_drill/dt).astype(int)
ax.plot(p_drill, np.arange(37), label='T_drill')


ax.set_title(
    f'Ch.{chNo}, pocessed data vs pulse position (est. delay = {round(buffer*10**3, 2)}ms) \n' +
    f'each segment = {round(N*dt*10**3, 2)}ms \n' +
    f'f = {round(fmin*10**-3)}...{round(fmax*10**-3)}kHz, smoothing window = {smoothing*10**3}ms'
)
#--- x-axis
ax.set_xlabel('t [ms]')
xticks = np.arange(0, N, 10000)
ax.set_xticks(xticks)
ax.set_xticklabels(np.round(dt* xticks*10**3, 2))
#--- y-axis
ax.set_ylabel('Segment No.')
ax.set_yticks(np.arange(0, 37, 5))
ax.set_yticklabels(np.arange(1, 38, 5))
# For gridding
ax.set_yticks(np.arange(0, 37), minor=True)
ax.grid(which='minor')
ax.grid(which='major')

plt.show()    
    

## Check the pulse of deep vs shallow penetration

In [ ]:
# Lower & upper bounds
q = 0.1
thres_low = np.quantile(d4, q)
thres_up = np.quantile(d4, 1.0-q+0.01)

In [ ]:
print(f'pulses < {thres_low} = {np.argwhere(d4<thres_low).flatten()}')
print(f'pulses > {thres_up} = {np.argwhere(d4>thres_up).flatten()}')

In [ ]:
# Segment & concatenate
def seg_concat(t0, obj, T, dt, offset, ret_indices=False):
    N = int(T/dt)
    arr = np.zeros((len(obj), N))
    indices = np.zeros((len(obj), 2), dtype=int)

    for row, t in enumerate(t0[obj]):
        # For trimming and aligning the signal segment 
        middle = t + buffer
        start = middle - offset #[s]
        #end = start + T # [s]
        idx_start = int(start/dt)
        idx_end = idx_start + N

        arr[row, :] = data_proc[np.newaxis, idx_start:idx_end]
        indices[row, :] = [idx_start, idx_end]
    
    if ret_indices == False:
        return arr
    else:
        return arr, indices


In [ ]:
T = 10.0*10**-3
offset = 1.0*10**-3

sig_shallow, idx_shallow = seg_concat(
    t0=t0, 
    obj=np.argwhere(d4<thres_low).flatten(), 
    T=T, 
    dt=dt,
    offset=offset,
    ret_indices=True
)
sig_deep, idx_deep = seg_concat(
    t0=t0, 
    obj=np.argwhere(d4>thres_up).flatten(), 
    T=T, 
    dt=dt,
    offset=offset,
    ret_indices=True
)
print(f'low = {sig_shallow.shape[0]}, upper = {sig_deep.shape[0]}')

In [ ]:
#%matplotlib widget
%matplotlib inline

fig, ax = plt.subplots(figsize=(12, 5))
# Entire duration
t = dt*np.arange(len(data_proc))
l1, = ax.plot(t, data_proc)

# # d4 < thres_low
# for (start, end) in zip(*idx_shallow.T):
#     l2, = ax.plot(t[start:end], data_proc[start:end], color='C1')
    
# # d4 > thres_up
# for (start, end) in zip(*idx_deep.T):
#     l3, = ax.plot(t[start:end], data_proc[start:end], color='C2')

# 37th pulse
start_37 = int((buffer + t0[-1]  - 10**-3)/dt)
end_37 = start_37 + int(T/dt)
l7, = ax.plot(t[start_37:end_37], data_proc[start_37:end_37], color='C1')

    
T_pulse = 10.4*10**-3 #[s]
# 49th pulse
start_49 = int((buffer + t0[-1] - 10**-3 + ((49-len(t0) + 1)*T_pulse))/dt)
end_49 = start_49 + int(T/dt)
l4, = ax.plot(t[start_49:end_49], data_proc[start_49:end_49], color='C3')

#---- outliers
# 67th pulse
start_67 = start_49 + int(((67 - 49 + 1)*T_pulse + 3.0*10**-3)/dt)
end_67 = start_67 + int(T/dt)
l5, = ax.plot(t[start_67:end_67], data_proc[start_67:end_67], color='C4')
# 76th pulse
start_76 = start_67 + int(((76 - 67 + 1)*T_pulse  - 10**-3)/dt)
end_76 = start_76 + int(T/dt)
l6, = ax.plot(t[start_76:end_76], data_proc[start_76:end_76], color='C4')
    

# Figure setting
ax.set_title(
    f'No.901 pulse welds ch.{chNo}: deep vs shallow drill \n' +
    f'f = {round(fmin*10**-3)}...{round(fmax*10**-3)}kHz'
)
ax.set_xlabel('t[s]')
ax.set_ylabel('amplitude')
ax.legend(
    [l1, l7, l4, l5],
    ['whole', '37th', '49th', '67th & 76th'], 
    loc="upper right"
)
ax.grid(True)

## Compare pre- and post-49th pulse 
Comment: <br>
Amplitude after 37th pulse is NOT easy to determine automatically, as we don't know the exact timing of the pulse 

In [ ]:
def array_indexing(vec, p_start, p_end):
    # p_end is excluded!!!
    # Duration 
    t_start = buffer + t0[-1] - 1.0*10**-3 + (p_start-len(t0) + 1)*T_pulse
    t_end = buffer + t0[-1] - 1.5* 10**-3 + (p_end-len(t0) + 1)*T_pulse
    
    # Base
    info = np.zeros((p_end - p_start, 3), dtype = int)
    
    # iterate over each estimated starting point
    for row, t in enumerate(np.arange(t_start, t_end, T_pulse)):
        # For trimming and aligning the signal segment 
        middle = t 
        start = middle - 10**-3 #[s]
        end = start + round(dt* N, 4) # [s]
        idx_start = int(start/dt)
        idx_end = idx_start + N
        
        # Update
        info[row, 0] = p_start + row
        info[row, 1] = idx_start
        info[row, 2] = idx_end
    
    return info


def array_formatting(vec, info):
    # base
    arr = np.zeros((info.shape[0], N))
    
    for row in range(info.shape[0]):
        start, end = info[row, 1:]
        arr[row] = data_proc[start:end]
    return arr



In [ ]:
T_pulse = 10.8*10**-3

# Indices for array formatting
info2 = array_indexing(data_proc, 38, 49)
info3 = array_indexing(data_proc, 49, 81)


fig, ax = plt.subplots(figsize=(12, 5))
# Entire duration
t = dt*np.arange(len(data_proc))
l1, = ax.plot(t, data_proc)

for start, end in zip(*info2[:, 1:].T):
    l2, = ax.plot(t[start:end], data_proc[start:end], color='C1')
    
for start, end in zip(*info3[:, 1:].T):
    l3, = ax.plot(t[start:end], data_proc[start:end], color='C2')

ax.grid(True)


In [ ]:
# Pulses up to 37th 
peak_set1 = np.max(arr, axis=1)

# pulses b/w 38...49
arr2 = array_formatting(data_proc, info2)
peak_set2 = np.max(arr2, axis=1)

# pulses after 49
arr3 = array_formatting(data_proc, info3)
peak_set3 = np.max(arr3, axis=1)
# remove the outlier pulses
#row_outliers = np.unique(np.argwhere(arr3 > 20)[:, 0])
#peak_set3 = np.delete(peak_set3, row_outliers)

print(f'Median peak pre-49th pulse = {np.median(np.concatenate((peak_set1, peak_set2)))}')
print(f'Median peak post-49th pulse = {np.median(peak_set3)}')


In [ ]:
#%matplotlib widget
%matplotlib inline

fig, (ax0, ax1, ax2) = plt.subplots(3, 1, figsize=(12, 10))

# x axis
t = 10**3*dt*np.arange(sig_shallow.shape[1]) #[ms]


# Shallow weld
ax0.set_title(f'd4 < {thres_low}mm (bottom {round(q*100)}%)')
ax0.plot(t, sig_shallow.T)
ax0.grid(True)
#ax0.set_xlabel('t[ms]')
ax0.set_ylabel('amplitude')
# for plotting the t- line
ax0.vlines(
    x = offset*10**3, 
    ymin=min(sig_shallow.min(), sig_deep.min())-0.2, 
    ymax=max(sig_shallow.max(), sig_deep.max())+0.2,
    colors='purple', label ='t0'
)
ax0.legend(loc='upper right')

# Shallow weld
ax1.set_title(f'd4 > {thres_up}mm (top {round(q*100)}%)')
ax1.plot(t, sig_deep.T)
ax1.grid(True)
#ax1.set_xlabel('t[ms]')
ax1.set_ylabel('amplitude')
# for plotting the t- line
ax1.vlines(
    x = offset*10**3, 
    ymin=min(sig_shallow.min(), sig_deep.min())-0.2,  
    ymax=max(sig_shallow.max(), sig_deep.max())+0.2,
    colors='purple', label ='t0'
)
ax1.legend(loc='upper right')

# Averaged signals
# Average signal of the shallow weld
ax2.plot(t, np.mean(sig_shallow, axis=0), label=f'd4 < {thres_low}mm (bottom {round(q*100)}%)')
# Average signal of deep weld
ax2.plot(t, np.mean(sig_deep, axis=0), label=f'd4 > {thres_up}mm (top {round(q*100)}%)')
# for plotting the t- line
ax2.vlines(
    x = offset*10**3, 
    ymin=min(sig_shallow.min(), sig_deep.min())-0.2, 
    ymax=max(sig_shallow.max(), sig_deep.max())+0.2,
    colors='purple', label ='t0'
)

# Tidy up
ax2.set_title('Averaged')
ax2.legend(loc='upper right')
ax2.grid(True)
ax2.set_xlabel('t[ms]')
ax2.set_ylabel('amplitude')

plt.suptitle(f'No. 901, Ch.{chNo} (f = {round(fmin*10**-3)}...{round(fmax*10**-3)}kHz)')
plt.show()

## Area summation

In [ ]:
# Parameters
#----- Convolution window related
T_win = 3*10**-3 #[s], window duration
N_win = int(T_win/dt)
# Modify the window duration s.t. the signal length is odd 
if N_win%2 == 0:
    N_win += 1
print(f'Window duration = {round(T_win*10**3, 2)}ms, length = {N_win}')

#----- Trimming the signal
T_sig = t4[-1] + T_win
# Modify the trimmed signal duration s.t. the signal length is odd 
N_sig = int(T_sig/dt)
if N_sig%2 == 0:
    N_sig += 1
    
print(f'Entire signal duration = {round(data.shape[0]*dt*10**3, 2)}ms')
print(f'Signal is trimmed: {round(T_sig*10**3, 2)}ms')
print(f'Trimmed signal length = {N_sig}')

In [ ]:
# Rectangular window to convolve
# Base
rect = np.zeros(N_sig)
# Put the ones in the middle
center = int((N_sig - 1)/2)
start = center - int((N_win - 1)/2)
end = start + N_win
rect[start:end] = np.ones(N_win)

# Normalize, s.t. area(rect) == 1
rect = rect / np.sqrt(N_win)
print(f'Area of the window = {np.sum(rect**2)}')

In [ ]:
# Convolution
y = scsig.convolve(
    in1=data_proc,
    in2=rect,
    mode='same'
)

In [ ]:
#%matplotlib widget
%matplotlib inline

plt.figure()
plt.title(f'Convolution with a rectanglar window of {round(T_win*10**3, 2)}ms')
plt.plot(dt*np.arange(len(y)), y)

plt.xlabel('t [s]')
plt.ylabel('amplitude')
plt.grid(True)

In [ ]:
# Foramtting an array
y_arr = pulse_array(y, N=N)
print(f'y_arr.shape = {y_arr.shape}')

In [ ]:
%matplotlib widget
#%matplotlib inline

# Setting
fig, (ax0, ax1) = plt.subplots(1, 2)
# Figure size
fig.set_figwidth(15)
fig.set_figheight(6)
plt.suptitle(
    f'Ch.{chNo}, sliding window area \n' + 
    f' area "width" = {round(T_win*10**3, 2)}ms \n' +
    f'f = {round(fmin*10**-3)}...{round(fmax*10**-3)}kHz, smoothing = {smoothing*10**3}ms'
)

#======== imshow
im = ax0.imshow(y_arr, aspect=N/37, cmap='rainbow')
#plt.colorbar(im)
# Add the t0 line
p_t0 = int(offset/dt) # pulse position 
ax0.vlines(
    x = p_t0, 
    ymin=-0.5, ymax=36.5,
    colors='red', label ='5.0'
)
# Add the T_drill line
p_drill = p_t0 + (T_drill/dt).astype(int)
ax0.plot(p_drill, np.arange(37), label='T_drill')

ax0.set_title('Heatmap view')
#--- x-axis
ax0.set_xlabel('t [ms]')
xticks = np.arange(0, N, 10000)
ax0.set_xticks(xticks)
ax0.set_xticklabels(np.round(dt* xticks*10**3, 2))
#--- y-axis
ax0.set_ylabel('Segment No.')
ax0.set_yticks(np.arange(0, 37, 5))
ax0.set_yticklabels(np.arange(1, 38, 5))
# For gridding
ax0.set_yticks(np.arange(0, 37), minor=True)
ax0.grid(which='minor')
ax0.grid(which='major')

#========== Compare the maximal amplitude of each row
idx_shallow = np.argwhere(d4<thres_low).flatten()
idx_deep = np.argwhere(d4>thres_up).flatten()

peaks_conv = np.max(y_arr, axis=1)
ax1.plot(np.arange(1, len(peaks_conv)+1), peaks_conv)
ax1.vlines(
    x = idx_shallow+1, 
    ymin=peaks_conv.min(), ymax=peaks_conv.max(),
    linestyle='dashed',
    colors='C1', label ='shallow'
)
ax1.vlines(
    x = idx_deep+1, 
    ymin=peaks_conv.min(), ymax=peaks_conv.max(),
    linestyle='dashed',
    colors='C2', label ='deep'
)

ax1.set_title('Peaks of each row (pulses)')
ax1.set_xlabel('pulse No.')
ax1.set_ylabel('Max(convolution)')
ax1.legend(loc='upper right')
ax1.grid('True')

plt.show()    

In [ ]:
print(f'pulses < {thres_low} = {np.argwhere(d4<thres_low).flatten()+1}')
print(f'pulses > {thres_up} = {np.argwhere(d4>thres_up).flatten()+1}')